# Import

In [1]:
import csv
import os
import pickle
import sys
from itertools import product

import numpy as np
import pandas as pd
from scipy import linalg, sparse
from sklearn.metrics import adjusted_mutual_info_score
from sklearn.mixture import GaussianMixture
from tqdm import tqdm

## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")
sys.path.append("../Function/")

In [3]:
from ALL import config 
from util import *
from extmath import row_norms

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [5]:
s3 = S3Manager()

In [6]:
data_type = "20NewsSampled1"  # sys.argv[1]
vectorize_type = "sentenceBERT"  # sys.argv[2]
transformer_model = "sentence-transformers/all-MiniLM-L6-v2"  # sys.argv[3]

In [7]:
vector_dims = config["vectorize"][vectorize_type][transformer_model]["dims"]
normalizations = config["vectorize"][vectorize_type][transformer_model]["normalization"]
vector_model_nums = config["vectorize"][vectorize_type][transformer_model]["max_model_num"]

model_nums = config["clustering"]["gmm"]["max_model_num"]
covariance_types = config["clustering"]["gmm"]["covariance_types"]
topic_nums = config["data"][data_type_classifier(data_type)]["class_num"]
depression_type = "umap"

# Read data

In [8]:
df_path = s3.download(f"Preprocessing/{data_type}/master.csv")

In [9]:
df = pd.read_csv(df_path[0], index_col=0)

In [10]:
labels_path = s3.download(f"Preprocessing/{data_type}/class.csv")

In [11]:
label = df["class"].to_numpy()

In [12]:
with open(labels_path[0], mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [13]:
if vectorize_type == "doc2vec":
    vectors_path = f"Clustering/{data_type}/{vectorize_type}/vector"
    models_path = f"Clustering/{data_type}/{vectorize_type}/GMM/model/"
elif vectorize_type == "sentenceBERT":
    vectors_path = f"Clustering/{data_type}/{vectorize_type}/{transformer_model}/vector"
    models_path = f"Clustering/{data_type}/{vectorize_type}/{transformer_model}/GMM/model/"
else:
    raise NotImplementedError

In [14]:
s3.download(vectors_path)

['/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/umap/128/centralized/0.npy',
 '/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/umap/128/normalized/0.npy',
 '/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/umap/16/centralized/0.npy',
 '/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/umap/16/normalized/0.npy',
 '/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/umap/2/centralized/0.npy',
 '/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/umap/2/normalized/0.npy',
 '/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/umap/256/centralized/0.npy',
 '/home/jovyan/temporary/Clustering/

In [15]:
s3.download(models_path)

['/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/128/centralized/10/diag/0.sav',
 '/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/128/centralized/10/diag/1.sav',
 '/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/128/centralized/10/diag/10.sav',
 '/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/128/centralized/10/diag/11.sav',
 '/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/128/centralized/10/diag/12.sav',
 '/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/128/centralized/10/diag/13.sav',
 '/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/128/centraliz

# functions

In [16]:
def _estimate_maharanobis_dist(X, means, precisions_chol, covariance_type):
    """Estimate the log Gaussian probability.
    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)
    means : array-like of shape (n_components, n_features)
    precisions_chol : array-like
        Cholesky decompositions of the precision matrices.
        'full' : shape of (n_components, n_features, n_features)
        'tied' : shape of (n_features, n_features)
        'diag' : shape of (n_components, n_features)
        'spherical' : shape of (n_components,)
    covariance_type : {'full', 'tied', 'diag', 'spherical'}
    Returns
    -------
    log_prob : array, shape (n_samples, n_components)
    """
    n_samples, n_features = X.shape
    n_components, _ = means.shape
    
    if covariance_type == "full":
        log_prob = np.empty((n_samples, n_components))
        for k, (mu, prec_chol) in enumerate(zip(means, precisions_chol)):
            y = np.dot(X, prec_chol) - np.dot(mu, prec_chol)
            log_prob[:, k] = np.sum(np.square(y), axis=1)

    elif covariance_type == "tied":
        log_prob = np.empty((n_samples, n_components))
        for k, mu in enumerate(means):
            y = np.dot(X, precisions_chol) - np.dot(mu, precisions_chol)
            log_prob[:, k] = np.sum(np.square(y), axis=1)

    elif covariance_type == "diag":
        precisions = precisions_chol**2
        log_prob = (
            np.sum((means**2 * precisions), 1)
            - 2.0 * np.dot(X, (means * precisions).T)
            + np.dot(X**2, precisions.T)
        )

    elif covariance_type == "spherical":
        precisions = precisions_chol**2
        log_prob = (
            np.sum(means**2, 1) * precisions
            - 2 * np.dot(X, means.T * precisions)
            + np.outer(row_norms(X, squared=True), precisions)
        )
    # Since we are using the precision of the Cholesky decomposition,
    # `- 0.5 * log_det_precision` becomes `+ log_det_precision_chol`
    return log_prob

In [17]:
def gmm_value(gmm, vectors, label):
    pred = gmm.predict(vectors)
    prob = np.exp(gmm._estimate_weighted_log_prob(vectors))
    dist = _estimate_maharanobis_dist(
        vectors, gmm.means_, gmm.precisions_cholesky_, gmm.covariance_type
    )
    aic = gmm.aic(vectors)
    bic = gmm.bic(vectors)
    mi = adjusted_mutual_info_score(pred, label)
    logl = gmm.score(vectors, label)
    return {
        "pred": pred,
        "prob": prob,
        "dist": dist,
        "aic": aic,
        "bic": bic,
        "mi": mi,
        "logl": logl,
    }

# Calculate Stats

In [18]:
if vectorize_type == "doc2vec":
    value_path = (
        f"/home/jovyan/temporary/Postprocessing/{data_type}/{vectorize_type}/GMM"
    )
if vectorize_type == "sentenceBERT":
    value_path = f"/home/jovyan/temporary/Postprocessing/{data_type}/{vectorize_type}/{transformer_model}/GMM"
else:
    raise NotImplementedError

In [43]:
for vector_model_num, vector_dim, normalization in tqdm(
    product(range(vector_model_nums), vector_dims, normalizations)
):
    vectors = np.load(
        f"{root_path_temporary}{vectors_path}/{depression_type}/{vector_dim}/{normalization}/{vector_model_num}.npy"
    )
    for covariance_type, topic_num in product(covariance_types, topic_nums):
        for model_num in range(model_nums):
            gmm = pickle.load(
                open(
                    f"{root_path_temporary}{models_path}{vector_dim}/{normalization}/{topic_num}/{covariance_type}/{model_num}.sav",
                    "rb",
                )
            )
            values = gmm_value(gmm, vectors, label)
            pred, prob, dist, *stat = values.items()
            # save
            for _name, _value in [pred, prob, dist]:
                save_path = f"{value_path}/{_name}/{vector_dim}/{normalization}/{vector_model_num}/{covariance_type}/{topic_num}/{model_num}.npy"
                np.save(make_filepath(save_path), _value)
            
            stat_path = f"{value_path}/stat/{vector_dim}/{normalization}/{vector_model_num}/{covariance_type}/{topic_num}/{model_num}.csv"
            with open(make_filepath(stat_path), "w") as f:
                writer = csv.DictWriter(f, dict(stat).keys())
                writer.writeheader()
                writer.writerow(dict(stat))
            

14it [09:15, 39.67s/it]


## upload file

In [44]:
s3.upload(value_path)

'/home/jovyan/temporary/Postprocessing/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM'

In [45]:
s3.delete_local_all()

/home/jovyan/temporary/Postprocessing/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM
/home/jovyan/temporary/Preprocessing/20NewsSampled1/master.csv
/home/jovyan/temporary/Preprocessing/20NewsSampled1/class.csv
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/umap/128/centralized/0.npy
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/umap/128/normalized/0.npy
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/umap/16/centralized/0.npy
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/umap/16/normalized/0.npy
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/vector/umap/2/centralized/0.npy
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6

/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/centralized/30/full/7.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/centralized/30/full/8.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/centralized/30/full/9.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/centralized/30/spherical/0.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/centralized/30/spherical/1.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/centralized/30/spherical/10.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/centralized/30/spherica

/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/normalized/30/full/6.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/normalized/30/full/7.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/normalized/30/full/8.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/normalized/30/full/9.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/normalized/30/spherical/0.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/normalized/30/spherical/1.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/256/normalized/30/spherical/10.sav
/hom

/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/10/full/7.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/10/full/8.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/10/full/9.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/10/spherical/0.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/10/spherical/1.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/10/spherical/10.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/10/spherica

/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/30/full/3.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/30/full/4.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/30/full/5.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/30/full/6.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/30/full/7.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/30/full/8.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/centralized/30/full/9.sav
/home/jovyan/

/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/normalized/10/full/14.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/normalized/10/full/15.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/normalized/10/full/2.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/normalized/10/full/3.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/normalized/10/full/4.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/normalized/10/full/5.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/384/normalized/10/full/6.sav
/home/jovyan/tempo

/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/4/normalized/20/full/2.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/4/normalized/20/full/20.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/4/normalized/20/full/21.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/4/normalized/20/full/22.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/4/normalized/20/full/23.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/4/normalized/20/full/24.sav
/home/jovyan/temporary/Clustering/20NewsSampled1/sentenceBERT/sentence-transformers/all-MiniLM-L6-v2/GMM/model/4/normalized/20/full/25.sav
/home/jovyan/temporary/Clust

In [46]:
send_line_notify(f"CalcStats.py {data_type} sentenceBERT {transformer_model}")

In [51]:
stats_lists = {
    model_num: {
        vector_dim: {
            covariance_type: {topic_num: dict() for topic_num in topic_nums}
            for covariance_type in covariance_types
        }
        for vector_dim in vector_dims
    }
    for model_num in range(model_nums)
}

In [74]:
for vector_model_num in range(vector_model_nums):
    for model_num in tqdm(range(model_nums)):
        for vector_dim in vector_dims:
            for covariance_type in covariance_types:
                for topic_num in topic_nums:
                    _df = pd.DataFrame(
                        stats_lists[model_num][vector_dim][covariance_type]
                    )
                    stats_df = pd.concat(
                        {
                            vector_dim: _df.loc[:, topic_num]
                            for vector_dim in vector_dims
                        },
                        axis=1,
                    ).T
                    stats_path = f"../data/{data_type}/{vectorize_type}/GMM/stats/{normalization}/{covariance_type}/{topic_num}/{model_num}.csv"
                    os.makedirs(os.path.dirname(stats_path), exist_ok=True)
                    stats_df.to_csv(stats_path)

100%|██████████| 10/10 [00:00<00:00, 40.82it/s]
